In [1]:
import json
import tweepy
import time
import configparser
from datetime import datetime
from kafka import KafkaProducer
from kafka.errors import KafkaError

In [2]:
class TweeterStreamListener(tweepy.StreamListener):
    
    def __init__(self, api):
        self.api = api
        super(tweepy.StreamListener, self).__init__()
        self.producer = KafkaProducer(bootstrap_servers='ip-172-31-19-70.ec2.internal:6667')

    def on_data(self, data):
        if 'text' not in json.loads(data):
            return
        msg = json.loads(data)[u'text']
        msg = json.dumps(msg).encode('utf-8')
        print(json.loads(data))
        try:
            self.producer.send('twitter', msg)
        except KafkaError:
            log.exception()
            pass
        except Exception as e:
            print(e)
            return False
        return True

    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False
        print("Error received in kafka producer")
        print(status_code)
        return True

    def on_timeout(self):
        return True 
    
if __name__ == '__main__':

    # Get the authorization keys
    config = configparser.ConfigParser()
    config.read_file(open('/home/ec2-user/datapipeline/stream/twitter-app-credentials'))
    consumer_key = 'z5lTf3sAbJDjGnGFsRFcvv1MK'
    consumer_secret = 'HbXFtsDHCuFllDI2MBhxjUThK0Yv96JuPzn3lf3aHmVNpZCI0V'
    access_key = '852055580361588736-vN0Fn0AjT7zCnRZ4Ljj7pQc4r06dtGd'
    access_secret = 'JZlVDeK9PDJhlj0ndJrDGLoYD9q4JmtZBBmZsLFYEE1V7'
    #consumer_key = config.defaults()['consumerkey']
    #consumer_secret = config.defaults()['consumersecret']
    #access_key = config.defaults()['accesstoken']
    #access_secret = config.defaults()['accesstokensecret']

    # Call twitter api using Tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
        
    # Start listening
    listener_twitter = TweeterStreamListener(api)
    stream = tweepy.Stream(auth, listener = listener_twitter)
    
    # Filter tweets
    stream.filter(track=['trump', 'healthcare'], languages = ['en'], async=True)

{'created_at': 'Mon Aug 07 16:33:37 +0000 2017', 'id': 894597408104906752, 'id_str': '894597408104906752', 'text': 'Mueller now doing Opposition Research to hurt President Trump in 2020 EVEN IF no crimes are found. Leaks will Flow from his Democratic Team.', 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1725560054, 'id_str': '1725560054', 'name': 'John S Hartman', 'screen_name': '1ltcjsh', 'location': None, 'url': None, 'description': None, 'protected': False, 'verified': False, 'followers_count': 22, 'friends_count': 261, 'listed_count': 0, 'favourites_count': 9, 'statuses_count': 400, 'created_at': 'Tue Sep 03 13:42:56 +0000 2013', 'utc_offset': -25200, 'time_zone': 'Arizona', 'geo_enabled': False, 'lang': 'en', 'contributors_enabled': False, 'is_translator'

{'created_at': 'Mon Aug 07 16:33:37 +0000 2017', 'id': 894597408159522816, 'id_str': '894597408159522816', 'text': 'See our latest #Lebanon, PA #job and click to apply: Pharmacist - Floater - https://t.co/RWPiZai55S #Healthcare #Hiring', 'source': '<a href="http://www.tweetmyjobs.com" rel="nofollow">TweetMyJOBS</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 57569071, 'id_str': '57569071', 'name': 'TMJ-PA Health Jobs', 'screen_name': 'tmj_pa_health', 'location': 'Pennsylvania', 'url': 'http://www.careerarc.com/job-seeker', 'description': 'Follow this account for geo-targeted Healthcare job tweets in Pennsylvania Non-Metro. Need help? Tweet us at @CareerArc!', 'protected': False, 'verified': False, 'followers_count': 592, 'friends_count': 302, 'listed_count': 200, 'favourites_count': 0, 'statuses_count': 1085, 'created_at': 'Fri Jul 17

{'created_at': 'Mon Aug 07 16:33:38 +0000 2017', 'id': 894597410378309632, 'id_str': '894597410378309632', 'text': 'RT @BigerguyTim: @chuckwoolery @BreitbartNews Brooks is a career pol . Has stood by nasty comments he made about Trump.', 'source': '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 703568996851777536, 'id_str': '703568996851777536', 'name': 'MJB', 'screen_name': 'MJBColorado', 'location': None, 'url': None, 'description': None, 'protected': False, 'verified': False, 'followers_count': 440, 'friends_count': 467, 'listed_count': 37, 'favourites_count': 16724, 'statuses_count': 40409, 'created_at': 'Sat Feb 27 13:14:55 +0000 2016', 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'lang': 'en', 'contributors_enabled': False, 'is_transla

{'created_at': 'Mon Aug 07 16:33:38 +0000 2017', 'id': 894597410869170176, 'id_str': '894597410869170176', 'text': '@SocratesNYTimes @chelseahandler What about the politics of liberal hate? It is their entire platform. Hate Trump,… https://t.co/E4VtmG9csO', 'display_text_range': [33, 140], 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'truncated': True, 'in_reply_to_status_id': 894198959304044544, 'in_reply_to_status_id_str': '894198959304044544', 'in_reply_to_user_id': 868991540231798784, 'in_reply_to_user_id_str': '868991540231798784', 'in_reply_to_screen_name': 'SocratesNYTimes', 'user': {'id': 887387968289296384, 'id_str': '887387968289296384', 'name': "Justin O'Connor", 'screen_name': 'WakeUpTheGiants', 'location': 'Upstate, NY', 'url': None, 'description': "'Sup Twitter. I'm here. The Triggering continues. Can't deal? Go back to living in the #victimhood.", 'protected': False, 'verified': False, 'followers_count': 57, 'friends_count': 330, 'liste

{'created_at': 'Mon Aug 07 16:33:38 +0000 2017', 'id': 894597411863109632, 'id_str': '894597411863109632', 'text': 'Neither served in Vietnam. #Blumenthal is a liar, but #Trump unnecessarily brings up his own past getting out of th… https://t.co/9NONgL59D0', 'display_text_range': [0, 140], 'source': '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>', 'truncated': True, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 48598570, 'id_str': '48598570', 'name': 'Sampa', 'screen_name': 'sp__sampa', 'location': 'São Paulo, Brasil', 'url': None, 'description': None, 'protected': False, 'verified': False, 'followers_count': 309, 'friends_count': 17, 'listed_count': 12, 'favourites_count': 63, 'statuses_count': 1186, 'created_at': 'Fri Jun 19 03:41:50 +0000 2009', 'utc_offset': -10800, 'time_zone': 'Brasilia', 'geo_enabled': False, 'lan

...

{'created_at': 'Mon Aug 07 16:33:44 +0000 2017', 'id': 894597438811422720, 'id_str': '894597438811422720', 'text': '@realDonaldTrump Nonpartisan @CBO analysis of @realDonaldTrump #budget concludes it doesn’t balance, as @POTUS clai… https://t.co/rr0waSjqbK', 'display_text_range': [17, 140], 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'truncated': True, 'in_reply_to_status_id': 894515865802223616, 'in_reply_to_status_id_str': '894515865802223616', 'in_reply_to_user_id': 25073877, 'in_reply_to_user_id_str': '25073877', 'in_reply_to_screen_name': 'realDonaldTrump', 'user': {'id': 4809408511, 'id_str': '4809408511', 'name': 'Forever Logical', 'screen_name': 'ForeverLogical', 'location': 'Washington, USA', 'url': 'http://foreverlogical.com', 'description': '#TheResistance 🇺🇸❄️🏳️\u200d🌈 Logic & Facts to Fight the Lies & Propaganda.', 'protected': False, 'verified': False, 'followers_count': 1210, 'friends_count': 2102, 'listed_count': 12, 'favourites_count

{'created_at': 'Mon Aug 07 16:33:45 +0000 2017', 'id': 894597440409612288, 'id_str': '894597440409612288', 'text': 'Just ask him 2 explain in detail what TPP actually entails. \nMore fascinating: He had NO clue what he was canceling… https://t.co/8rG8E5SD7g', 'display_text_range': [0, 140], 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'truncated': True, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 31040699, 'id_str': '31040699', 'name': 'Network Sommelier', 'screen_name': 'wadvisor', 'location': 'USA', 'url': 'http://www.slideshare.net/wadvisor/how-family-offices-are-disrupting-the-future-of-investing-the-sea', 'description': 'Interested in #FamilyOffice deal flow, #Tech, #Healthcare, #Impact Investing, #Blockchain & #Bitcoin. ALL tweets are MY own views', 'protected': False, 'verified': False, 'followers_count': 1358, 'fr

{'created_at': 'Mon Aug 07 16:33:45 +0000 2017', 'id': 894597442267525120, 'id_str': '894597442267525120', 'text': 'RT @CitizenSlant: Trump Attacks Democratic Sen. Richard Blumenthal: ‘He was a Phony Vietnam Con Artist’ https://t.co/clbwIIJuZ8… https://t.…', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 3050904856, 'id_str': '3050904856', 'name': 'Ben Turner', 'screen_name': 'BLTurner816', 'location': 'Georgia', 'url': None, 'description': 'Follower of Christ. Husband. Outdoorsman. Dog lover.', 'protected': False, 'verified': False, 'followers_count': 69, 'friends_count': 95, 'listed_count': 127, 'favourites_count': 6407, 'statuses_count': 6543, 'created_at': 'Sat Feb 21 21:39:56 +0000 2015', 'utc_offset': -39600, 'time_zone': 'International 

{'created_at': 'Mon Aug 07 16:33:45 +0000 2017', 'id': 894597442527797248, 'id_str': '894597442527797248', 'text': 'RT @HouseJudDems: 🆕 @RepCummings, @RepJohnConyers, Rep. Connolly, @RepCicilline Request Documents about Trump’s Regulatory Task Force… ', 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 791095997124358144, 'id_str': '791095997124358144', 'name': 'Thisguy🍻', 'screen_name': 'BeernBrownsGuy', 'location': 'Georgia, USA', 'url': None, 'description': "Dad, Husband, Die hard Browns fan, beach lover, beer enthusiast, Bourbon drinker, Mechanical Engineer, liberal in a red state.  Motto: Don't be a d*ck. #Resist", 'protected': False, 'verified': False, 'followers_count': 312, 'friends_count': 676, 'listed_count': 0, 'favourites_count': 26286, 'statuses_count

{'created_at': 'Mon Aug 07 16:33:45 +0000 2017', 'id': 894597443194585089, 'id_str': '894597443194585089', 'text': 'RT @yashar: WATCH: @kayleighmcenany left CNN yesterday....today, she made her debut on Trump TV sharing "the real news". https://t.co/joqxN…', 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1286072497, 'id_str': '1286072497', 'name': 'Raylon Randle', 'screen_name': 'thatboyraylon', 'location': 'Camden, Arkansas', 'url': None, 'description': 'UofA Neurosurgeon in the making :)', 'protected': False, 'verified': False, 'followers_count': 1007, 'friends_count': 2275, 'listed_count': 2, 'favourites_count': 96260, 'statuses_count': 4436, 'created_at': 'Thu Mar 21 14:07:58 +0000 2013', 'utc_offset': None, 'time_zone': None, 'geo_enabled